# © Artur Czarnecki. All rights reserved.

## Runtime Integration — STATIC + DYNAMIC (end-to-end smoke tests)

This notebook validates **runtime-level integration**:

`RuntimeEngine.run()` → `PipelineFactory` → `PlannerStaticPipeline` / `PlannerDynamicPipeline`

Key rules:
- One code cell = one independent test (no shared state).
- Only real framework components (no fakes).
- Deterministic planning via `ScriptedPlanSource` to prove the runtime path reliably.


## Test RT-S1 — Runtime STATIC_PLAN smoke (deterministic plan source)

Goal:
- Run `RuntimeEngine.run()` with `step_planning_strategy=STATIC_PLAN`
- Planning is deterministic via `ScriptedPlanSource([FINALIZE])`
- Assert:
  - non-empty answer
  - route.strategy is set
  - debug_trace contains runtime `run_start` and `run_end` events


In [2]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))


from intergrax.llm_adapters.llm_provider import LLMProvider
from intergrax.llm_adapters.llm_provider_registry import LLMAdapterRegistry

from intergrax.runtime.drop_in_knowledge_mode.config import RuntimeConfig, StepPlanningStrategy
from intergrax.runtime.drop_in_knowledge_mode.engine.runtime_context import RuntimeContext
from intergrax.runtime.drop_in_knowledge_mode.engine.runtime import RuntimeEngine

from intergrax.runtime.drop_in_knowledge_mode.responses.response_schema import RuntimeRequest

from intergrax.runtime.drop_in_knowledge_mode.planning.plan_sources import PlanSpec, ScriptedPlanSource
from intergrax.runtime.drop_in_knowledge_mode.planning.engine_plan_models import PlannerPromptConfig
from intergrax.runtime.drop_in_knowledge_mode.planning.plan_loop_models import PlanLoopPolicy
from intergrax.runtime.drop_in_knowledge_mode.planning.step_planner import StepPlannerConfig
from intergrax.runtime.drop_in_knowledge_mode.planning.step_executor_models import StepExecutorConfig

from intergrax.runtime.drop_in_knowledge_mode.session.in_memory_session_storage import InMemorySessionStorage
from intergrax.runtime.drop_in_knowledge_mode.session.session_manager import SessionManager


async def _run_rt_s1() -> None:
    # 1) Deterministic plan: FINALIZE
    plan_finalize = PlanSpec(
        json_obj={
            "version": "1",
            "intent": "generic",
            "next_step": "finalize",
            "reasoning_summary": "Deterministic STATIC runtime test plan (FINALIZE).",
            "ask_clarifying_question": False,
            "clarifying_question": None,
            "use_websearch": False,
            "use_user_longterm_memory": False,
            "use_rag": False,
            "use_tools": False,
        }
    )
    plan_source = ScriptedPlanSource([plan_finalize])

    # 2) Real LLM adapter (same pattern as previous notebooks)
    llm_adapter = LLMAdapterRegistry.create(LLMProvider.OLLAMA)

    # 3) Runtime config (STATIC requires planner_prompt_config)
    config = RuntimeConfig(
        llm_adapter=llm_adapter,
        enable_rag=False,
        enable_websearch=False,
        tools_mode="off",

        step_planning_strategy=StepPlanningStrategy.STATIC_PLAN,

        step_planner_cfg=StepPlannerConfig(),
        step_executor_cfg=StepExecutorConfig(),

        planner_prompt_config=PlannerPromptConfig(),
        plan_loop_policy=PlanLoopPolicy(max_replans=2, max_same_plan_repeats=2),

        # Deterministic plan injection
        plan_source=plan_source,
    )

    # 4) Context + engine
    session_manager = SessionManager(storage=InMemorySessionStorage())
    ctx = RuntimeContext.build(config=config, session_manager=session_manager)
    engine = RuntimeEngine(context=ctx)

    # 5) Request
    req = RuntimeRequest(
        session_id="rt-s1-static-session",
        user_id="rt-s1-static-user",
        message="RT-S1: return a short answer confirming STATIC runtime integration works.",
        attachments=[],
    )

    # 6) Run
    answer = await engine.run(req)

    # 7) Assertions (minimal, trustworthy)
    assert answer.answer is not None and answer.answer.strip() != "", "Expected a non-empty final answer."
    assert answer.route is not None, "Expected RouteInfo to be present."
    assert isinstance(answer.route.strategy, str) and answer.route.strategy.strip() != "", "Expected route.strategy to be set."

    events = (answer.debug_trace or {}).get("events", [])
    assert any(e.get("step") == "run_start" for e in events), "Missing run_start in debug_trace.events."
    assert any(e.get("step") == "run_end" for e in events), "Missing run_end in debug_trace.events."

    print("RT-S1 OK")
    print("Answer length:", len(answer.answer))
    print("Route.strategy:", answer.route.strategy)
    print("Debug trace tail:", events[-3:])


await _run_rt_s1()


AssertionError: Missing run_start in debug_trace.events.